In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [6]:
dataset=dataset.drop("User ID",axis=1)

In [7]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,True
1,35,20000,0,True
2,26,43000,0,False
3,27,57000,0,False
4,19,76000,0,True
...,...,...,...,...
395,46,41000,1,False
396,51,23000,1,True
397,50,20000,1,False
398,36,33000,0,True


In [8]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [9]:
independent=dataset[[ 'Age', 'EstimatedSalary','Gender_Male']]

In [10]:
independent

,Age,EstimatedSalary,Gender_Male
0,19,19000,True
1,35,20000,True
2,26,43000,False
3,27,57000,False
4,19,76000,True
...,...,...,...
395,46,41000,False
396,51,23000,True
397,50,20000,False
398,36,33000,True


In [11]:
dependent=dataset[['Purchased']]

In [12]:
dependent

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=1/3,random_state=0)

In [14]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [31]:
from sklearn.tree import DecisionTreeClassifier

In [33]:
from sklearn.model_selection import GridSearchCV
param_grid= {'criterion':['gini','entropy'],
            'max_features':['sqrt','log2',None],
             'splitter':['best','random']}
grid=GridSearchCV(DecisionTreeClassifier(),param_grid,refit=True,verbose=3,n_jobs=1,scoring='f1_weighted') 
grid.fit(x_train,y_train)           

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5] END criterion=gini, max_features=sqrt, splitter=best;, score=0.830 total time=   0.0s
[CV 2/5] END criterion=gini, max_features=sqrt, splitter=best;, score=0.849 total time=   0.0s
[CV 3/5] END criterion=gini, max_features=sqrt, splitter=best;, score=0.833 total time=   0.0s
[CV 4/5] END criterion=gini, max_features=sqrt, splitter=best;, score=0.832 total time=   0.0s
[CV 5/5] END criterion=gini, max_features=sqrt, splitter=best;, score=0.847 total time=   0.0s
[CV 1/5] END criterion=gini, max_features=sqrt, splitter=random;, score=0.826 total time=   0.0s
[CV 2/5] END criterion=gini, max_features=sqrt, splitter=random;, score=0.834 total time=   0.0s
[CV 3/5] END criterion=gini, max_features=sqrt, splitter=random;, score=0.738 total time=   0.0s
[CV 4/5] END criterion=gini, max_features=sqrt, splitter=random;, score=0.796 total time=   0.0s
[CV 5/5] END criterion=gini, max_features=sqrt, splitter=random;, score=0.88

GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['sqrt', 'log2', None],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [35]:
re=grid.cv_results_
grid_predictions=grid.predict(x_test)

                   

In [37]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_predictions)

In [39]:
print(cm)

[[76  9]
 [ 8 41]]


In [41]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_predictions)

In [43]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.90      0.89      0.90        85
           1       0.82      0.84      0.83        49

    accuracy                           0.87       134
   macro avg       0.86      0.87      0.86       134
weighted avg       0.87      0.87      0.87       134



In [45]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'criterion': 'entropy', 'max_features': None, 'splitter': 'best'}: 0.8733997218497793


In [47]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

0.8654261704681873

In [49]:
table=pd.DataFrame.from_dict(re)

In [51]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.004382,0.008763,0.016449,0.002956,gini,sqrt,best,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.829615,0.849057,0.833323,0.832483,0.847020,0.838299,0.008072,6
1,0.001815,0.002248,0.007006,0.001998,gini,sqrt,random,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.826263,0.834012,0.738451,0.796284,0.885265,0.816055,0.048229,12
2,0.004334,0.006116,0.002003,0.004006,gini,log2,best,"{'criterion': 'gini', 'max_features': 'log2', ...",0.765716,0.849057,0.833323,0.851527,0.868632,0.833651,0.035765,8
3,0.004113,0.006066,0.009182,0.008845,gini,log2,random,"{'criterion': 'gini', 'max_features': 'log2', ...",0.839696,0.831253,0.740173,0.851527,0.905069,0.833544,0.053293,9
4,0.003364,0.004128,0.014080,0.002051,gini,None,best,"{'criterion': 'gini', 'max_features': None, 's...",0.808927,0.868752,0.833323,0.870362,0.943041,0.864881,0.045345,2
5,0.000000,0.000000,0.006251,0.007656,gini,None,random,"{'criterion': 'gini', 'max_features': None, 's...",0.847141,0.852030,0.796284,0.869709,0.885265,0.850086,0.030099,4
6,0.006252,0.007657,0.003126,0.006252,entropy,sqrt,best,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.804764,0.813511,0.813179,0.832483,0.924528,0.837693,0.044357,7
7,0.003864,0.005717,0.006877,0.003551,entropy,sqrt,random,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.847141,0.831253,0.758030,0.808775,0.905069,0.830054,0.048085,10
8,0.004741,0.006289,0.004721,0.006268,entropy,log2,best,"{'criterion': 'entropy', 'max_features': 'log2...",0.826263,0.850809,0.758030,0.851527,0.905069,0.838340,0.047703,5
9,0.003126,0.006252,0.006253,0.007658,entropy,log2,random,"{'criterion': 'entropy', 'max_features': 'log2...",0.864871,0.828959,0.740173,0.868632,0.826499,0.825827,0.046268,11
